In [0]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import re

In [0]:
def human2machine(number):
  number = re.sub(r'[, \$]', r'', str(number))
  #if re.match(r'million', number):
  if 'million' in number:
    number = re.sub(r'^([0-9\.]+).*$', r'\1 * 1000000', number)
  elif 'billion' in number:
    number = re.sub(r'^([0-9\.]+).*$', r'\1 * 1000000000', number)
  elif 'trillion' in number:
    number = re.sub(r'^([0-9\.]+).*$', r'\1 * 1000000000000', number)
    
  try:
    result = int(eval(number))
  except ValueError, NameError:
    result = 'Something went wrong'
  
  return result

In [0]:
budget_url = 'https://www.cia.gov/library/publications/the-world-factbook/fields/2056.html'
popula_url = 'https://www.cia.gov/library/publications/the-world-factbook/rankorder/2119rank.html'

In [0]:
budget_soup = BeautifulSoup(urllib2.urlopen(budget_url).read(), 'html.parser')
popula_soup = BeautifulSoup(urllib2.urlopen(popula_url).read(), 'html.parser')

In [0]:
budget_table = budget_soup.find('table', id='fieldListing')
popula_table = popula_soup.find('table', id='rankOrder')  

In [0]:
 budgets = []

for country in budget_table.find_all('tr')[1:]:
  country_name = country.find_all('td', 'country')[0].text
  data = str(country.find_all('td', 'fieldData')[0].text).split('\n')[1:]
  revenues = re.sub(r'^.*: (\$.*(illion|00)|\$?NA).*$', r'\1', data[0])
  if 'NA' in revenues: revenues = None
  expenditures = re.sub(r'^.*: (\$.*(illion|00)|\$?NA).*$', r'\1', data[1])
  if 'NA' in expenditures: expenditures = None
  budgets.append((str(country_name), revenues, expenditures))

In [7]:
df_budgets = pd.DataFrame(budgets, columns=['Country', 'Revenues_HumanReadable', 'Expenditures_HumanReadable'])
df_budgets.head()

,Country,Revenues_HumanReadable,Expenditures_HumanReadable
0,Afghanistan,$1.992 billion,$6.636 billion
1,Albania,$3.486 billion,$3.765 billion
2,Algeria,$52.08 billion,$70.74 billion
3,American Samoa,$241.2 million,$243.7 million
4,Andorra,$1.872 billion,$2.06 billion


In [0]:
# Later we will merge Gaza with West Bank
df_budgets.iloc[75, 1] = 0
df_budgets.iloc[75, 2] = 0

df_budgets = df_budgets.dropna()

In [0]:
population = []

for country in popula_table.find_all('tr')[1:]:
  country_name = country.find_all('td')[1].text
  pop_number = country.find_all('td')[2].text
  population.append((country_name, pop_number))

In [10]:
df_population = pd.DataFrame(population, columns=['Country', 'Population_HumanReadable'])
df_population.head()

,Country,Population_HumanReadable
0,China,"1,379,302,771"
1,India,"1,281,935,911"
2,United States,"326,625,791"
3,Indonesia,"260,580,739"
4,Brazil,"207,353,391"


In [11]:
df = pd.merge(df_budgets, df_population, how='left', on='Country')
df.head()

,Country,Revenues_HumanReadable,Expenditures_HumanReadable,Population_HumanReadable
0,Afghanistan,$1.992 billion,$6.636 billion,"34,124,811"
1,Albania,$3.486 billion,$3.765 billion,"3,047,987"
2,Algeria,$52.08 billion,$70.74 billion,"40,969,443"
3,American Samoa,$241.2 million,$243.7 million,"51,504"
4,Andorra,$1.872 billion,$2.06 billion,"85,702"


In [12]:
df[df['Country'] == 'World']

,Country,Revenues_HumanReadable,Expenditures_HumanReadable,Population_HumanReadable
225,World,$20.35 trillion,$22.74 trillion,NaN


In [0]:
iworld = df[df['Country'] == 'World'].index
df.iloc[iworld, 3] = 0
df['Revenues'] = df['Revenues_HumanReadable'].apply(human2machine)
df['Expenditures'] = df['Expenditures_HumanReadable'].apply(human2machine)
df['Population'] = df['Population_HumanReadable'].apply(human2machine)
df.iloc[iworld, 6] = sum(df['Population'])
df.iloc[iworld, 3] = re.sub(r'(\d{1,3}(?=(\d{3})+(?!\d)))',r'\1,', str(df.iloc[225, 6]))

In [14]:
df[df['Country'] == 'World']

,Country,Revenues_HumanReadable,Expenditures_HumanReadable,Population_HumanReadable,Revenues,Expenditures,Population
225,World,$20.35 trillion,$22.74 trillion,"7,404,266,749",20350000000000,22740000000000,7404266749


In [15]:
df[ (df['Country'] == 'Gaza Strip') | (df['Country'] == 'West Bank') ]

,Country,Revenues_HumanReadable,Expenditures_HumanReadable,Population_HumanReadable,Revenues,Expenditures,Population
73,Gaza Strip,0,0,"1,795,183",0,0,1795183
224,West Bank,$1.314 billion,$1.278 billion,"2,747,943",1314000000,1278000000,2747943


In [16]:
# Little messy here, we copy the values of West Bank, then we merge the population with Gaza strip

palestine = df.iloc[224].tolist()
palestine[0] = 'Palestine'
df.loc[len(df)] = palestine
pseries = df.iloc[-1:]
pseries.iloc[0, 6] = df['Population'][ (df['Country'] == 'Gaza Strip') | (df['Country'] == 'West Bank') ].sum()
pseries.iloc[0, 3] = re.sub(r'(\d{1,3}(?=(\d{3})+(?!\d)))',r'\1,', str(pseries.iloc[0, 6]))

df = df.drop([73, 224])
df = df.sort_values(by='Country')
df.reset_index(inplace=True, drop=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
df.head()

,Country,Revenues_HumanReadable,Expenditures_HumanReadable,Population_HumanReadable,Revenues,Expenditures,Population
0,Afghanistan,$1.992 billion,$6.636 billion,"34,124,811",1992000000,6636000000,34124811
1,Albania,$3.486 billion,$3.765 billion,"3,047,987",3486000000,3765000000,3047987
2,Algeria,$52.08 billion,$70.74 billion,"40,969,443",52080000000,70740000000,40969443
3,American Samoa,$241.2 million,$243.7 million,"51,504",241200000,243700000,51504
4,Andorra,$1.872 billion,$2.06 billion,"85,702",1872000000,2060000000,85702


In [0]:
# Separator as semi colon, as comma is already in the data
df.to_csv('countries_by_budget_and_population.csv', sep=';')